In [ ]:
from dotenv import load_dotenv

import clears
import sheets

load_dotenv()

In [ ]:
gc = sheets.authenticate_gspread()

In [ ]:
cc = sheets.load_current_clears_from_main_sheet(gc)

In [ ]:
state, map_difficulties = clears.get_current_state_and_maps_from_sheet_values(cc)

In [ ]:
map_difficulties

In [ ]:
maps = {map_name for (player_name, map_name), clear_type in state.items()}

In [ ]:
splits = [m.replace("\n", " ").replace("  ", " ").split(" by ") for m in maps]
set([len(s) for s in splits])

In [ ]:
map_clear_types = {}
for map_name, author in splits:
    if map_name.endswith("]"):
        i = map_name.rfind("[")
        clear_type = map_name[i:]
        map_name = map_name[:i - 1]
        if map_name not in map_clear_types:
            map_clear_types[map_name] = set()
        map_clear_types[map_name].add(clear_type)
    else:
        if map_name not in map_clear_types:
            map_clear_types[map_name] = set()
        map_clear_types[map_name].add("[C]")
for map_name, clear_types in sorted(map_clear_types.items()):
    print(map_name, "-", clear_types)

In [ ]:
set.union(*map_clear_types.values())

so there's only 3 "clear types": c, fc, and "all maps" (which is only for devil's den)

In [ ]:
import helpers
from collections import defaultdict

player_clears = defaultdict(set)
map_clearers = defaultdict(set)

for (player_name, map_name), new_val in state.items():
    player_clears[player_name].add(map_name)
    map_clearers[map_name].add(player_name)

clear_diffs_by_player_and_map = defaultdict(set)

for new_key in state.keys():
    player_name, map_name = new_key

    new_val = state.get(new_key, "")
    map_difficulty = map_difficulties[map_name]
    trimmed_map_name, clear_type = helpers.trim_map_name(map_name)

    clear_diffs_by_player_and_map[(player_name, trimmed_map_name)].add((clear_type, new_val, map_difficulty))

In [ ]:
c = defaultdict(set)
for (p, m), x in clear_diffs_by_player_and_map.items():
    for t, v, d in x:
        c[(t, v)].add(m)